# PatientInsights : Advancing Patient Care through Enhanced Digital Record Analytics

## BA820 Final Project

#### Group 14 - Jishnu Moorty , Shivakumar Vinod Pillai , Yashvardhan Singh Ranawat

## Project Overview

### Problem Statement 
An estimated 80% of healthcare data is unstructured. The most common sources being clinical notes containing discharge summaries, lab reports , progress notes etc. Translating this data in a structured manner poses a unique challenge due to its inconsistent nature. The same concept can be articulated in numerous ways across different organizations and fields of specialty. Having a complete picture of this information at hand can lead to better care for patients and NLP combined with unsupervised learning can unlock the valuable insights trapped in unstructured data.


### Dataset & Data Source 
Medical Information Mart for Intensive Care (MIMIC-III) Clinical Database. Please find the [link](https://mimic.mit.edu/docs/iii/) here.

### Business Relevance 
- Risk Assessment: Streamlining manual workflows by identifying clinical concepts that indicate complicated conditions, to accurately characterize a patient’s risk 
- Quality Reporting : Converting unstructured data to provide a more accurate picture of a patient's health status while identifying gaps in care or documentation recorded. .
- Clinical Decision Support: Clustering can assist in helping providers make informed decisions through comparing similar cases and their outcomes. 
- Research and Knowledge Discovery: Researchers can uncover new associations and risk factors by analyzing clusters of patients with similar characteristics.


### Data Dictionary 

### Importing All Libraries

In [1]:
import re
from nltk.corpus import  stopwords
import spacy
import nltk
nltk.download('stopwords')

### Connecting to BigQuery

In [2]:
from google.cloud import bigquery

client = bigquery.Client()

### Loading Data

This dataset was accesible through bigquery, Since only one member was provided access to use this data, we moved the tables to our own project's dataset. Original dataset is called 'physionet-data' on bigquery

### Note events Table 

##### Notes associated with hospital stays.

| Column      | Type      | Definition                                                                                   |
|-------------|-----------|----------------------------------------------------------------------------------------------|
| row_id      | int4      | Unique row identifier.                                                                       |
| subject_id  | int4      | Foreign key. Identifies the patient.                                                         |
| hadm_id     | int4      | Foreign key. Identifies the hospital stay.                                                   |
| chartdate   | timestamp | Date when the note was charted.                                                              |
| charttime   | timestamp | Date and time when the note was charted. Notes without a time have NULL in this column.      |
| storetime   | timestamp |                                                                                              |
| category    | varchar   | Category of the note, e.g., Discharge summary.                                               |
| description | varchar   | A more detailed categorization for the note, sometimes entered by free-text.                 |
| cgid        | int4      | Foreign key. Identifies the caregiver.                                                       |
| iserror     | bpchar    | Flag to highlight an error with the note.                                                    |
| text        | text      | Content of the note.                                                                         |



In [3]:
noteevents_query = ('''SELECT *
                FROM `ba820-unsup-ml.mimic3.noteevents`''')

noteevents_data = client.query(noteevents_query).to_dataframe()

noteevents_data.head(2)

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,460342,63621,124508,2134-06-23,2134-06-23 12:18:00,2134-06-23 12:18:24,Nutrition,Clinical Nutrition Note,19139,<NA>,"Subjective Per patient , no bloating or abd di..."
1,334680,30198,147913,2131-07-12,2131-07-12 09:01:00,2131-07-12 09:01:15,Nutrition,Clinical Nutrition Note,20157,<NA>,Potential for nutrition risk. Patient being mo...


### Patients Data

##### Patients associated with an admission to the ICU.

| Column      | Type       | Definition                                                                                   |
|-------------|------------|----------------------------------------------------------------------------------------------|
| row_id      | int4       | Unique row identifier.                                                                       |
| subject_id  | int4       | Primary key. Identifies the patient.                                                         |
| gender      | varchar    | Gender.                                                                                      |
| dob         | timestamp  | Date of birth.                                                                               |
| dod         | timestamp  | Date of death. Null if the patient was alive at least 90 days post hospital discharge.       |
| dod_hosp    | timestamp  | Date of death recorded in the hospital records.                                              |
| dod_ssn     | timestamp  | Date of death recorded in the social security records.                                       |
| expire_flag | int4       | Flag indicating that the patient has died.                                                   |

In [4]:
patient_query = ('''SELECT *
                FROM `ba820-unsup-ml.mimic3.patients`''')

patient_data = client.query(patient_query).to_dataframe()

patient_data.head(2)

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,46038,98139,F,2035-02-19,2116-05-24,2116-05-24,2116-05-24,1
1,156,166,F,1805-01-23,2105-01-24,2105-01-24,2105-01-24,1


### Admissions Data

##### Hospital admissions associated with an ICU stay.

| Column               | Type      | Definition                                                                                   |
|----------------------|-----------|----------------------------------------------------------------------------------------------|
| row_id               | int4      | Unique row identifier.                                                                       |
| subject_id           | int4      | Foreign key. Identifies the patient.                                                         |
| hadm_id              | int4      | Primary key. Identifies the hospital stay.                                                   |
| admittime            | timestamp | Time of admission to the hospital.                                                           |
| dischtime            | timestamp | Time of discharge from the hospital.                                                         |
| deathtime            | timestamp | Time of death.                                                                               |
| admission_type       | varchar   | Type of admission, for example emergency or elective.                                        |
| admission_location   | varchar   | Admission location.                                                                          |
| discharge_location   | varchar   | Discharge location                                                                           |
| insurance            | varchar   | Insurance type.                                                                              |
| language             | varchar   | Language.                                                                                    |
| religion             | varchar   | Religion.                                                                                    |
| marital_status       | varchar   | Marital status.                                                                              |
| ethnicity            | varchar   | Ethnicity.                                                                                   |
| edregtime            | timestamp |                                                                                              |
| edouttime            | timestamp |                                                                                              |
| diagnosis            | varchar   | Diagnosis.                                                                                   |
| hospital_expire_flag | int2      |                                                                                              |
| has_chartevents_data | int2      | Hospital admission has at least one observation in the CHARTEVENTS table.                    |


In [5]:
admissions_query = ('''SELECT *
                FROM `ba820-unsup-ml.mimic3.admissions`''')

admissions_data = client.query(admissions_query).to_dataframe()

admissions_data.head(2)

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,40230,41420,143206,2147-01-06 16:57:00,2147-01-08 18:30:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,ICF,Medicare,ENGL,JEWISH,WIDOWED,WHITE,2147-01-06 13:30:00,2147-01-06 17:36:00,ACUTE SUBDURAL HEMATOMA,0,1
1,45890,58899,177890,2102-04-04 08:12:00,2102-04-10 16:00:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,ICF,Medicare,ENGL,NOT SPECIFIED,SINGLE,WHITE,2102-04-04 06:46:00,2102-04-04 08:33:00,THREATENED LEFT LOWER EXTREMITY,0,1


In [6]:
noteevents_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2083180 entries, 0 to 2083179
Data columns (total 11 columns):
 #   Column       Dtype         
---  ------       -----         
 0   ROW_ID       Int64         
 1   SUBJECT_ID   Int64         
 2   HADM_ID      Int64         
 3   CHARTDATE    dbdate        
 4   CHARTTIME    datetime64[us]
 5   STORETIME    datetime64[us]
 6   CATEGORY     object        
 7   DESCRIPTION  object        
 8   CGID         Int64         
 9   ISERROR      Int64         
 10  TEXT         object        
dtypes: Int64(5), datetime64[us](2), dbdate(1), object(3)
memory usage: 184.8+ MB


### EDA

### Pre-processing Text Data

##### Removing Punctuations

In [7]:
def remove_punc(column):
    punc_pattern = r'[^\w\s]'
    column = re.sub(punc_pattern,"", column)
    return column
    

noteevents_data['text_cleaned'] = noteevents_data['TEXT'].apply(remove_punc)

noteevents_data[['TEXT','text_cleaned']].head(2)

,TEXT,text_cleaned
0,"Subjective Per patient , no bloating or abd di...",Subjective Per patient no bloating or abd dis...
1,Potential for nutrition risk. Patient being mo...,Potential for nutrition risk Patient being mon...


In [8]:
def remove_newlines(column):
    newline_pattern = r'\n'  
    return re.sub(newline_pattern, "", column)

noteevents_data['text_cleaned'] = noteevents_data['text_cleaned'].apply(remove_newlines)

noteevents_data[['TEXT','text_cleaned']].head(2)

,TEXT,text_cleaned
0,"Subjective Per patient , no bloating or abd di...",Subjective Per patient no bloating or abd dis...
1,Potential for nutrition risk. Patient being mo...,Potential for nutrition risk Patient being mon...


In [9]:
def remove_common_words(column):
    pattern = r'\b(?:First Name|Last Name|NamePattern1|firstname|lastname|name|TITLE)\b'
    return re.sub(pattern, "", column)

noteevents_data['text_cleaned'] = noteevents_data['text_cleaned'].apply(remove_common_words)

noteevents_data[['TEXT','text_cleaned']].head(2)

,TEXT,text_cleaned
0,"Subjective Per patient , no bloating or abd di...",Subjective Per patient no bloating or abd dis...
1,Potential for nutrition risk. Patient being mo...,Potential for nutrition risk Patient being mon...


In [10]:
def remove_abbreviation(column):
    # Pattern to match 'Dr.' optionally followed by a space
    pattern = r'\bDr\.\s?'
    return re.sub(pattern, "", column)


noteevents_data['text_cleaned'] = noteevents_data['text_cleaned'].apply(remove_abbreviation)

noteevents_data[['TEXT','text_cleaned']].head(2)

,TEXT,text_cleaned
0,"Subjective Per patient , no bloating or abd di...",Subjective Per patient no bloating or abd dis...
1,Potential for nutrition risk. Patient being mo...,Potential for nutrition risk Patient being mon...


In [ ]:
def remove_stopwords(text, language='english'):

    words = re.split(r'\s+', text)
    stop_words = set(stopwords.words(language))
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

noteevents_data['text_cleaned'] = noteevents_data['text_cleaned'].apply(remove_stopwords)

noteevents_data[['TEXT','text_cleaned']].head(2)

In [ ]:
# Load the English language model
nlp = spacy.load("en_core_web_sm")

def tokenize_text(text):
    doc = nlp(text)
    #word tokens
    words = [token.text for token in doc]
    # Sentence tokens
    sentences = [sent.text for sent in doc.sents]
    return words, sentences

# Apply the tokenize_text function and split the results into two columns
noteevents_data[['words', 'sentences']] = noteevents_data['text_cleaned'].apply(lambda x: pd.Series(tokenize_text(x)))

noteevents_data[['words','sentences']].head(2)

In [ ]:
import pandas as pd
with pd.option_context('display.max_colwidth', None):
    display(noteevents_data[['TEXT','text_cleaned']].head(5))